# 静态图和动态图

[![下载Notebook](https://gitee.com/mindspore/docs/raw/tutorials-develop/resource/_static/logo_notebook.png)](https://obs.dualstack.cn-north-4.myhuaweicloud.com/mindspore-website/notebook/tutorials-develop/tutorials/zh_cn/mindspore_pynative_mode_and_graph_mode.ipynb)&emsp;
[![下载样例代码](https://gitee.com/mindspore/docs/raw/tutorials-develop/resource/_static/logo_download_code.png)](https://obs.dualstack.cn-north-4.myhuaweicloud.com/mindspore-website/notebook/tutorials-develop/tutorials/zh_cn/mindspore_pynative_mode_and_graph_mode.py)&emsp;
[![查看源文件](https://gitee.com/mindspore/docs/raw/tutorials-develop/resource/_static/logo_source.png)](https://gitee.com/mindspore/docs/blob/tutorials-develop/tutorials/source_zh_cn/advance/pynative_mode_and_graph_mode.ipynb)

目前主流的深度学习框架有静态图和动态图两种执行模式。

- 静态图模式下，程序在编译执行时先生成神经网络的图结构，然后再执行图中涉及的计算操作。因此，在静态图模式下，编译器可以通过使用图优化等技术来获得更好的执行性能，有助于规模部署和跨平台运行。

- 动态图模式下，程序按照代码的编写顺序逐行执行，在执行正向过程中根据反向传播的原理，动态生成反向执行图。这种模式下，编译器将神经网络中的各个算子逐一下发执行，方便用户编写和调试神经网络模型。

## MindSpore静态图

在MindSpore中，静态图模式又被称为Graph模式，比较适合网络固定且需要高性能的场景，可以通过`context.set_context(mode=context.GRAPH_MODE)`来设置成静态图模式。在静态图模式下，基于图优化、计算图整图下沉等技术，编译器可以针对图进行全局的优化，因此在静态图下能获得较好的性能，但是同时也存在一些特殊的语法约束，详细信息可参考[静态图语法支持](https://www.mindspore.cn/docs/note/zh-CN/master/static_graph_syntax_support.html)。

### Graph模式执行原理

在Graph模式下，MindSpore通过源码转换的方式，将Python的源码转换成IR，并在此基础上进行图优化，最终在硬件设备上执行优化后的图。MindSpore使用的是一种基于图表示的函数式IR，即MindIR，采用了接近于ANF函数式的语义。Graph模式是基于MindIR进行编译优化的，使用Graph模式时，需要使用[nn.Cell](https://mindspore.cn/docs/api/zh-CN/master/api_python/nn/mindspore.nn.Cell.html#mindspore.nn.Cell)类并且在`construct`函数中编写执行代码，或者调用`ms_function`装饰器。

### 在静态图模式下使用`nn.Cell`类

Graph模式的代码用例如下所示：

In [2]:
import numpy as np
import mindspore.nn as nn
import mindspore.ops as ops
from mindspore import context, Tensor

# 设置运行模式为静态图模式
context.set_context(mode=context.GRAPH_MODE)

class Net(nn.Cell):
    def __init__(self):
        super(Net, self).__init__()
        # 使用矩阵相乘算子
        self.mul = ops.Mul()

    # 定义类的执行代码
    def construct(self, x, y):
        return self.mul(x, y)

x = Tensor(np.array([1.0, 2.0, 3.0]).astype(np.float32))
y = Tensor(np.array([4.0, 5.0, 6.0]).astype(np.float32))

net = Net()
# 打印张量x和张量y的乘积
print(net(x, y))

[ 4. 10. 18.]


### 使用`ms_function`装饰器

#### 1. 修饰独立函数

使用`ms_function`装饰器时，可以对独立定义的函数进行修饰，使其在静态图模式下运行，示例如下：

In [3]:
import numpy as np
import mindspore.ops as ops
from mindspore import context, Tensor, ms_function

# 设置运行模式为动态图模式
context.set_context(mode=context.PYNATIVE_MODE)

# 使用ms_function装饰器，指定add_func函数在静态图模式下执行
@ms_function
def add_func(x, y):
    return ops.add(x, y)

x = Tensor(np.array([1.0, 2.0, 3.0]).astype(np.float32))
y = Tensor(np.array([4.0, 5.0, 6.0]).astype(np.float32))
# 张量x和张量相加
out = add_func(x, y)
# 打印结果
print(out)

[5. 7. 9.]


在上面的示例代码中，虽然一开始设置了运行模式为动态图模式，但是由于使用了`ms_function`装饰器对函数`add_func(x, y)`进行了修饰，所以函数`add_func(x, y)`仍然是以静态图模式运行。

#### 2. 修饰Cell的成员函数

使用ms_function装饰器时，可以对Cell的成员函数进行修饰，示例代码如下：

In [4]:
import numpy as np
import mindspore.nn as nn
import mindspore.ops as ops
from mindspore import context, Tensor, ms_function

# 设置运行模式为动态图模式
context.set_context(mode=context.PYNATIVE_MODE)

class Add(nn.Cell):
    # 使用ms_function装饰器，指定construct函数在静态图模式下执行
    @ms_function
    def construct(self, x, y):
        out = x + y
        return out

x = Tensor(np.array([1.0, 2.0, 3.0]).astype(np.float32))
y = Tensor(np.array([4.0, 5.0, 6.0]).astype(np.float32))

net = Add()
# 打印结果
print(net(x, y))

[5. 7. 9.]


在上面的示例代码中，虽然一开始设置了运行模式为动态图模式，但是由于使用了`ms_function`装饰器对类`Add`的成员函数`construct(self, x, y)`进行了修饰，所以函数`construct(self, x, y)`仍然是以静态图模式运行。

### Graph模式自动微分原理

在MindSpore中，Graph模式下的自动微分原理可以参考[自动微分](https://www.mindspore.cn/tutorials/zh-CN/master/autograd.html)。

## MindSpore动态图

在MindSpore中，动态图模式又被称为PyNative模式，可以通过`context.set_context(mode=context.PYNATIVE_MODE)`来设置成动态图模式。在脚本开发和网络流程调试中，推荐使用动态图模式进行调试，支持执行单算子、普通函数和网络、以及单独求梯度的操作。

### PyNative模式执行原理

在PyNative模式下，用户可以使用完整的Python API，此外针对使用MindSpore提供的API时，框架会根据用户选择的硬件平台（Ascend，GPU，CPU）或环境信息，将算子API的操作在对应的硬件平台上执行，并返回相应的结果。框架整体的执行过程如下：

![process](https://gitee.com/mindspore/docs/raw/tutorials-develop/tutorials/source_zh_cn/advance/images/framework.png)

通过前端的Python API，调用到框架层，最终到相应的硬件设备上进行计算。下面我们通过一段简单的示例代码来进行说明：

In [5]:
import numpy as np
import mindspore.context as context
import mindspore.nn as nn
from mindspore import Tensor
import mindspore.ops as ops

# 设置运行模式为动态图模式
context.set_context(mode=context.PYNATIVE_MODE)

x = Tensor(np.array([1.0, 2.0, 3.0]).astype(np.float32))
y = Tensor(np.array([4.0, 5.0, 6.0]).astype(np.float32))

output = ops.add(x, y)
# 打印张量x和张量y的和
print(output.asnumpy())

[5. 7. 9.]


在上面的示例代码中，当调用到Python接口`ops.add(x, y)`时，会将Python的接口调用通过Pybind11调用到MindSpore框架的C++层，转换成C++的调用，接着框架会根据安装的MindSpore环境信息自动选择对应的硬件设备，在该硬件设备上执行add操作。

从上述原理可以看到，在PyNative模式下，Python脚本代码会根据Python的语法进行执行，而执行过程中涉及到MindSpore的API，会根据用户的设置在不同的硬件上执行，从而进行性能加速。因此，在PyNative模式下，用户可以随意使用Python的语法以及调试方法。例如可以使用常见的PyCharm、VS Code等IDE进行代码的调试。

### PyNative模式自动微分原理

在PyNative下，执行正向过程完全是按照Python的语法执行的，而反向传播过程是基于Tensor实现的。因此，我们在执行正向过程中，将所有应用于Tensor的操作记录下来，并针对每个操作求取其反向，然后将所有反向过程串联起来形成整体反向传播图（简称反向图），最终将反向图在设备上执行并计算出梯度。下面我们通过一段简单的示例代码说明PyNative模式自动微分原理。

对矩阵x乘上固定参数z，然后与y进行矩阵乘法，最终对x进行求导，代码如下：

In [6]:
import numpy as np
import mindspore.nn as nn
import mindspore.ops as ops
from mindspore import Tensor, context
from mindspore import Parameter
from mindspore import dtype as mstype

# 设置运行模式为动态图模式
context.set_context(mode=context.PYNATIVE_MODE)

class Net(nn.Cell):
    def __init__(self):
        super(Net, self).__init__()
        # 使用矩阵相乘算子
        self.matmul = ops.MatMul()
        # 定义参数z，值为2.0
        self.z = Parameter(Tensor(np.array([1.0], np.float32)), name='z')

    def construct(self, x, y):
        # x先乘以z，再和y相乘
        x = x * self.z
        out = self.matmul(x, y)
        return out

# 定义对x的求导
class GradNetWrtX(nn.Cell):
    def __init__(self, net):
        super(GradNetWrtX, self).__init__()
        self.net = net
        self.grad_op = ops.GradOperation()

    def construct(self, x, y):
        gradient_function = self.grad_op(self.net)
        return gradient_function(x, y)

x = Tensor([[0.8, 0.6, 0.2], [1.8, 1.3, 1.1]], dtype=mstype.float32)
y = Tensor([[0.11, 3.3, 1.1], [1.1, 0.2, 1.4], [1.1, 2.2, 0.3]], dtype=mstype.float32)
output = GradNetWrtX(Net())(x, y)
print(output)

[[4.51      2.7       3.6000001]
 [4.51      2.7       3.6000001]]


![forward](https://gitee.com/mindspore/docs/raw/tutorials-develop/tutorials/source_zh_cn/advance/images/forward.png) ![backward](https://gitee.com/mindspore/docs/raw/tutorials-develop/tutorials/source_zh_cn/advance/images/backward.png)

根据上述PyNative下构图原理可以看到，在正向传播过程中，我们记录了Mul的计算过程，根据Mul对应的反向bprop的定义，得到了反向的MulGrad算子，根据Mul算子的bprop定义，如下：

```Python
from mindspore.ops._grad.grad_base import bprop_getters

@bprop_getters.register(ops.Mul)
def get_bprop_mul(self):
    """Grad definition for `Mul` operation."""
    mul_func = P.Mul()

    def bprop(x, y, out, dout):
        bc_dx = mul_func(y, dout)
        bc_dy = mul_func(x, dout)
        return binop_grad_common(x, y, bc_dx, bc_dy)

    return bprop
```

可以看到对Mul的输入求反向，需要两个输入和输出的反向传播梯度值，此时根据实际的输入值，可以将z连接到MulGrad。以此类推，对下一个算子Matmul，相应的得到MatmulGrad信息，再根据bprop的输入输出，将上下文梯度传播连接起来。

最终，对于初始传播，在MindSpore中使用[sens](https://mindspore.cn/docs/api/zh-CN/master/api_python/ops/mindspore.ops.GradOperation.html?highlight=gradoperation#mindspore.ops.GradOperation)进行缩放，默认值为1。同理对于输入y求导，可以使用同样的过程进行推导。

### PyNative模式下的控制流

在MindSpore中，针对控制流语法并没有做特殊处理，直接按照Python的语法展开执行，进而对展开的执行算子进行自动微分操作。例如，对于for循环，在PyNative下会根据具体的循环次数，不断的执行for循环中的语句，并对其算子进行自动微分操作。

## 动静统一

### 概述

当前在业界支持动态图和静态图两种模式，动态图通过解释执行，具有动态语法亲和性，表达灵活；静态图使用jit编译优化执行，偏静态语法，在语法上有较多限制。动态图和静态图的编译流程不一致，语法约束不一致。MindSpore针对动态图和静态图模式，首先统一API表达，在两种模式下使用相同的API；其次统一动态图和静态图的底层微分机制。

![dynamic](https://gitee.com/mindspore/docs/raw/tutorials-develop/tutorials/source_zh_cn/advance/images/dynamic.png)

### 动态图和静态图互相转换

在MindSpore中，我们可以通过控制模式输入参数`context.set_context(mode=context.PYNATIVE_MODE)`来切换执行使用动态图还是静态图。由于在静态图下，对于Python语法有所限制，因此从动态图切换成静态图时，需要符合静态图的语法限制，才能正确使用静态图来进行执行。更多静态图的语法限制可以参考[静态图语法支持](https://www.mindspore.cn/docs/note/zh-CN/master/static_graph_syntax_support.html)。

### 动静结合

MindSpore支持在动态图下使用静态编译的方式来进行混合执行，通过使用`ms_function`装饰符来修饰需要用静态图来执行的函数对象，即可实现动态图和静态图的混合执行。示例代码如下：

In [7]:
import numpy as np
import mindspore.nn as nn
from mindspore import Tensor
from mindspore import ms_function
from mindspore import Parameter
import mindspore.context as context

class AddSelf(nn.Cell):
    # 通过ms_function修饰符指定construct函数按照静态图模式执行
    @ms_function
    def construct(self, x):
        x = x + x
        return x

class MulSelf(nn.Cell):
    def construct(self, x):
        x = x * x
        return x

context.set_context(mode=context.PYNATIVE_MODE)
input = Tensor(np.array([1.0, 2.0, 3.0]).astype(np.float32))
add_net = AddSelf()
mul_net = MulSelf()
print(add_net(input))
print(mul_net(input))

[2. 4. 6.]
[1. 4. 9.]


在使用ms_function来修饰函数，加速执行效率时，请注意以下几点：

1. ms_function修饰的函数须在静态图编译支持的语法范围内，包括但不限于数据类型等。

2. ms_function修饰的函数所支持的控制流语法，与静态图保持一致。其中，仅对固定循环次数或者分支条件的控制流结构具有加速效果。

3. 在PyNative模式下使用ms_function功能时，非ms_function修饰的部分支持断点调试；被ms_function修饰的部分由于是以静态图的方式编译，不支持断点调试。

4. 由于ms_function修饰的函数将按照静态图的方式编译执行，因此ms_function不支持修饰的函数中含有Hook算子，以及不支持修饰自定义Bprop函数等。

5. ms_function修饰的函数会受到静态图函数副作用的影响。函数副作用指：当调用函数时，除了函数返回值之外，还对主调用函数产生的附加影响，例如修改全局变量（函数外的变量），修改函数的参数等。

场景1：

In [8]:
import numpy as np
from mindspore import context, Tensor, ms_function

# pylint: disable=W0612

value = 5

@ms_function
def func(x, y):
    out = x + y
    value = 1
    return out

context.set_context(mode=context.PYNATIVE_MODE)
x = Tensor(np.array([1.0, 2.0, 3.0]).astype(np.float32))
y = Tensor(np.array([1.0, 2.0, 3.0]).astype(np.float32))
func(x, y)
print(value)

5


该场景下，`value`是全局变量且在`func`函数中被修改。此时，如果用`ms_function`修饰`func`函数，全局变量`value`的值将不会被修改。原因是：**静态图编译时，会优化掉与返回值无关的语句**。

场景2：

In [9]:
import numpy as np
import mindspore.nn as nn
from mindspore import context, Tensor, ms_function

class Func(nn.Cell):
    def __init__(self):
        super(Func, self).__init__()
        self.value = 5

    @ms_function
    def construct(self, x):
        out = self.value + x
        return out

context.set_context(mode=context.PYNATIVE_MODE)
x = Tensor(np.array([1.0, 2.0, 3.0]).astype(np.float32))
func = Func()
print("out1:", func(x))
func.value = 1
print("out2:", func(x))

out1: [6. 7. 8.]
out2: [6. 7. 8.]


从上面的打印可以看出，在修改了Func类的成员变量value的值为1之后，对成员函数construct的操作并无影响。这是因为在此场景下，用ms_function修饰了`Func`对象的`construct`成员函数，执行`Func`时将会以静态图的方式编译执行。由于静态图会缓存编译结果，第二次调用`Func`时，对`value`的修改不会生效。

### JIT Fallback

JIT Fallback是为了实现动静统一提出的功能特性。通过JIT Fallback等特性，静态图可以支持尽量多的动态图语法，使得静态图提供接近动态图的语法使用体验。

JIT Fallback是从静态图的角度出发考虑静态图和动态图的统一。MindSpore默认使用静态图模式，用户编写程序时需要遵循MindSpore[静态图语法支持](https://www.mindspore.cn/docs/note/zh-CN/master/static_graph_syntax_support.html)，语法使用存在约束限制。在动态图模式下，Python脚本代码会根据Python语法进行执行，用户可以使用任意Python语法。可以看出，静态图和动态图的语法约束限制是不同的。JIT Fallback特性可以使得静态图支持尽量多的动态图语法，用户能够灵活地进行静态图和动态图的切换。

当前JIT Fallback支持静态图模式的部分常量场景，包括在construct/ms_function中调用第三方库、创建及使用Tensor、调用Python的print打印等。更多JIT Fallback的说明和使用，请参考[JIT Fallback文档](https://www.mindspore.cn/tutorials/zh-CN/master/compiler_optimization/jit_fallback.html)。

代码用例如下，其中，MindSpore静态图模式不支持在construct中调用NumPy第三方库和创建Tensor对象，因此用例中的`x = np.array([1, 2, 3])`和`y = Tensor(x)`将会通过JIT Fallback特性使用Python解释器进行解释执行，从而实现对这些语法的支持。

In [ ]:
import numpy as np
import mindspore.nn as nn
from mindspore import context, Tensor

context.set_context(mode=context.GRAPH_MODE)

class Net(nn.Cell):
    def construct(self):
        x = np.array([1, 2, 3])
        y = Tensor(x)
        return y

net = Net()
print(net())